In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

path = "../data/naver-map-results-preprocessed.json"
save_path = "../data/naver-map-results-postprocessed-coord.json"
coord_path = "../data/old_addr_with_coordinates.csv"

In [ ]:
# JSON 파일 열기
with open(path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 데이터 출력
print(len(data))

In [ ]:
import math

# 좌표 데이터에서 위도와 경도 추출
coords = coord_data[['lat', 'lng']].values

# NaN 값 제거
coords = coords[~np.isnan(coords).any(axis=1)]  # NaN이 있는 행 제거

# NearestNeighbors 모델 생성
nbrs = NearestNeighbors(n_neighbors=3).fit(coords)

# 두 지점 간의 실제 거리 계산 함수 (Haversine 공식을 사용)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반지름 (km)
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # km 단위로 반환

# 각 데이터에 대해 가장 가까운 관광명소 찾기
for entry in tqdm(data):
    # entry에서 좌표 추출 (예: entry['location'])
    location = data[entry]['coordinate']  # 예시로 location 키 사용
    nearest_attractions = coord_data.to_dict(orient='records')
    
    # 각 관광명소와의 거리 계산
    for attraction in nearest_attractions:
        attraction['distance'] = haversine(location['lat'], location['lng'], attraction['lat'], attraction['lng'])  # 실제 거리 계산
    # 거리 기준으로 정렬하여 가장 가까운 3개 관광명소 선택
    nearest_attractions = sorted(nearest_attractions, key=lambda x: x['distance'])[:3]
    
    data[entry]['nearest_attractions'] = nearest_attractions

In [ ]:
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)